In [41]:
import pandas as pd
import numpy as np
import pickle

In [42]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn import *


In [ ]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("spanish_energy_price_prediction")

In [44]:
# Reading the data in a pandas dataframe
df = pd.read_csv("../data/energy_dataset.csv")

In [45]:
df['time'].nunique()


35064

In [46]:
df.describe()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
count,35045.000000,35046.000000,35046.0,35046.000000,35046.000000,35045.000000,35046.0,35046.0,35046.0,0.0,...,35045.000000,35046.0,35046.000000,35064.000000,0.0,35064.000000,35064.000000,35028.000000,35064.000000,35064.000000
mean,383.513540,448.059208,0.0,5622.737488,4256.065742,298.319789,0.0,0.0,0.0,NaN,...,269.452133,0.0,5464.479769,1439.066735,NaN,5471.216689,28712.129962,28696.939905,49.874341,57.884023
std,85.353943,354.568590,0.0,2201.830478,1961.601013,52.520673,0.0,0.0,0.0,NaN,...,50.195536,0.0,3213.691587,1677.703355,NaN,3176.312853,4594.100854,4574.987950,14.618900,14.204083
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,...,0.000000,0.0,0.000000,0.000000,NaN,237.000000,18105.000000,18041.000000,2.060000,9.330000
25%,333.000000,0.000000,0.0,4126.000000,2527.000000,263.000000,0.0,0.0,0.0,NaN,...,240.000000,0.0,2933.000000,69.000000,NaN,2979.000000,24793.750000,24807.750000,41.490000,49.347500
50%,367.000000,509.000000,0.0,4969.000000,4474.000000,300.000000,0.0,0.0,0.0,NaN,...,279.000000,0.0,4849.000000,576.000000,NaN,4855.000000,28906.000000,28901.000000,50.520000,58.020000
75%,433.000000,757.000000,0.0,6429.000000,5838.750000,330.000000,0.0,0.0,0.0,NaN,...,310.000000,0.0,7398.000000,2636.000000,NaN,7353.000000,32263.250000,32192.000000,60.530000,68.010000
max,592.000000,999.000000,0.0,20034.000000,8359.000000,449.000000,0.0,0.0,0.0,NaN,...,357.000000,0.0,17436.000000,5836.000000,NaN,17430.000000,41390.000000,41015.000000,101.990000,116.800000


In [47]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,time,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_coal-derived_gas,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_fossil_oil_shale,generation_fossil_peat,generation_geothermal,...,generation_waste,generation_wind_offshore,generation_wind_onshore,forecast_solar_day_ahead,forecast_wind_offshore_eday_ahead,forecast_wind_onshore_day_ahead,total_load_forecast,total_load_actual,price_day_ahead,price_actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [48]:
df = df.drop('time', axis=1)

In [49]:
#Deleting the empty columns
df.interpolate(method='linear', limit_direction='forward', inplace=True, axis=0)

In [50]:
df = df.drop(['generation_hydro_pumped_storage_aggregated'], axis=1)
df = df.drop(['forecast_wind_offshore_eday_ahead'], axis=1)

In [51]:
df.dtypes

generation_biomass                             float64
generation_fossil_brown_coal/lignite           float64
generation_fossil_coal-derived_gas             float64
generation_fossil_gas                          float64
generation_fossil_hard_coal                    float64
generation_fossil_oil                          float64
generation_fossil_oil_shale                    float64
generation_fossil_peat                         float64
generation_geothermal                          float64
generation_hydro_pumped_storage_consumption    float64
generation_hydro_run-of-river_and_poundage     float64
generation_hydro_water_reservoir               float64
generation_marine                              float64
generation_nuclear                             float64
generation_other                               float64
generation_other_renewable                     float64
generation_solar                               float64
generation_waste                               float64
generation

In [52]:
df.isnull().sum()

generation_biomass                             0
generation_fossil_brown_coal/lignite           0
generation_fossil_coal-derived_gas             0
generation_fossil_gas                          0
generation_fossil_hard_coal                    0
generation_fossil_oil                          0
generation_fossil_oil_shale                    0
generation_fossil_peat                         0
generation_geothermal                          0
generation_hydro_pumped_storage_consumption    0
generation_hydro_run-of-river_and_poundage     0
generation_hydro_water_reservoir               0
generation_marine                              0
generation_nuclear                             0
generation_other                               0
generation_other_renewable                     0
generation_solar                               0
generation_waste                               0
generation_wind_offshore                       0
generation_wind_onshore                        0
forecast_solar_day_a

In [53]:
X=df.drop('price_actual',axis=1)
y=df['price_actual']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

# print dimension of predictors train set
print("The shape of X_train is:",X_train.shape)

# print dimension of predictors test set
print("The shape of X_test is:",X_test.shape)

# print dimension of target train set
print("The shape of y_train is:",y_train.shape)

# print dimension of target test set
print("The shape of y_test is:",y_test.shape)

The shape of X_train is: (26298, 25)
The shape of X_test is: (8766, 25)
The shape of y_train is: (26298,)
The shape of y_test is: (8766,)


In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [56]:
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression()

In [57]:
y_pred_lin = lin_model.predict(X_test)

In [58]:
# rsquared_lin = lin_model.score(X_train,y_train)
rmse_lin = mean_squared_error(y_test, y_pred_lin)

In [59]:
print(rmse_lin)

86.36496749297201


In [60]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((lin_model), f_out)

In [61]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [62]:
y_pred_rf = rf_model.predict(X_test)

In [63]:
rmse_rf = mean_squared_error(y_test, y_pred_rf)

In [64]:
print(rmse_rf)

31.987714616498973


In [65]:
from xgboost import XGBRegressor

In [66]:



xgbr=XGBRegressor()


xgbr.fit(X_train,y_train)

y_pred_xgb = xgbr.predict(X_test)
rmse_xgb = mean_squared_error(y_test, y_pred_xgb)
print(rmse_xgb)


33.196487081104856


In [67]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = XGBRegressor(
        max_depth=int(params['max_depth']),   # Ensure it's an integer
        learning_rate=params['learning_rate'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        min_child_weight=params['min_child_weight'],
        objective=params['objective'],
        seed=params['seed']
        )
        booster.fit(X_train,y_train)
        y_pred = booster.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 3, 15, 1)),  # Reduced to [3, 15]
    'learning_rate': hp.loguniform('learning_rate', -3, -1),     # Reduced range to [0.05, 0.37]
    'reg_alpha': hp.loguniform('reg_alpha', -4, -2),             # Reduced range to [0.018, 0.135]
    'reg_lambda': hp.loguniform('reg_lambda', -4, -2),           # Reduced range to [0.018, 0.135]
    'min_child_weight': hp.loguniform('min_child_weight', -1, 2),# Reduced range to [0.367, 7.39]
    'objective': 'reg:squarederror',  # Updated to modern objective name
    'seed': 42
}
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=30,
    trials=Trials()
)



  3%|▎         | 1/30 [00:05<02:38,  5.47s/trial, best loss: 5.675784880610745]

c:\Users\loges\.virtualenvs\spanish_energy_price_predictor-_QYZrIZm\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



  7%|▋         | 2/30 [00:27<06:59, 14.97s/trial, best loss: 5.027609200868368]

c:\Users\loges\.virtualenvs\spanish_energy_price_predictor-_QYZrIZm\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



 10%|█         | 3/30 [00:38<06:01, 13.37s/trial, best loss: 5.027609200868368]

...

c:\Users\loges\.virtualenvs\spanish_energy_price_predictor-_QYZrIZm\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



 97%|█████████▋| 29/30 [05:48<00:15, 15.93s/trial, best loss: 4.906354436716539]

c:\Users\loges\.virtualenvs\spanish_energy_price_predictor-_QYZrIZm\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



100%|██████████| 30/30 [06:05<00:00, 12.19s/trial, best loss: 4.906354436716539]


c:\Users\loges\.virtualenvs\spanish_energy_price_predictor-_QYZrIZm\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [71]:
print(mlflow.__version__)
client = mlflow.tracking.MlflowClient()
experiments = client.search_experiments()
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}, Name: {experiment.name}")

2.16.2
Experiment ID: 2, Name: spanish_energy_price_prediction
Experiment ID: 1, Name: nyc-taxi-experiment
Experiment ID: 0, Name: Default
